# Data Wrangling Project: Wrangle and Analys Data

- ## Intorduction

In this project I'm  going to apply the data wrangling techniques that I've learned on some data that I will collect from Twitter using tweepy API, and after I'm done wrangling the data I will analyse it and create a visualization to help understand how data wrangling can affect the final outcome for any analysis.

I will follow the same steps that I learned in the data wrangling lesson to reach my goal, the steps are:

* Gathering the data
* Assessing the data
* Cleaning the data

after I'm done with these steps I will start analysing the cleaned data and compare it to the unclean data analysis.

=====================================================================================================================

## Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import tweepy as tw
import json

## Gathering

For the gathering we will accuire three types of data:

1. CSV data from the "Twitter-Archive-Enhanced"
2. TSV data generated from the Image prediction algorithim
3. JSON data from the Tweepy API

In [3]:
# Start with the csv file using read_csv function.
df_csv = pd.read_csv('twitter-archive-enhanced.csv')
df_csv.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
# Read the tsv file also using read_csv function but changing the sep variable
df_tsv = pd.read_csv("image-predictions.tsv",sep='\t')
df_tsv.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [5]:
# Authenticating and using the Tweepy API

consumer_key = '1ifPTGNs26JtJqirQFlyd4Jz1'
consumer_secret = 'ZR9382UxmSWgmwXX1PocpsyWHniotQYotMw1N8kJc0Pz4aG449'
access_token = '786162021561532417-waB9mIK8Ov8YcCQZJyAzg7yKz5QsFD5'
access_token_secret = 'SY145NwAH9m78MGhmIQW9gLh7hN7S4aUHNkwpMaEoJQhs'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

# Storing the Tweet IDs to use them with the API
tweet_ids = df_csv['tweet_id'].values

In [6]:
# Checking the user for the twitter account that we're intrested in
user = api.get_user('dog_rates')
print(user.name)
print(user.description)

WeRateDogs™
Your Only Source For Professional Dog Ratings Instagram and Facebook ➪ WeRateDogs partnerships@weratedogs.com ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀


In [7]:
with open('tweet_json.txt', 'w') as file:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, file)
            file.write('\n')
        except tw.TweepError:
            print('Failed to return json object for tweet ID: {}'.format(tweet_id))

Failed to return json object for tweet ID: 888202515573088257
Failed to return json object for tweet ID: 873697596434513921
Failed to return json object for tweet ID: 872668790621863937
Failed to return json object for tweet ID: 872261713294495745
Failed to return json object for tweet ID: 869988702071779329
Failed to return json object for tweet ID: 866816280283807744
Failed to return json object for tweet ID: 861769973181624320
Failed to return json object for tweet ID: 856602993587888130
Failed to return json object for tweet ID: 851953902622658560
Failed to return json object for tweet ID: 845459076796616705
Failed to return json object for tweet ID: 844704788403113984
Failed to return json object for tweet ID: 842892208864923648
Failed to return json object for tweet ID: 837366284874571778
Failed to return json object for tweet ID: 837012587749474308
Failed to return json object for tweet ID: 829374341691346946
Failed to return json object for tweet ID: 827228250799742977
Failed t

In [21]:
df_json = pd.read_json('tweet-json.txt', lines = True,encoding='utf-8')
df_json.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,NaN,2019-07-07 09:55:25,"{'hashtags': [{'text': 'weratedogs', 'indices'...","{'media': [{'id': 1147806019839565826, 'id_str...",0,False,NaN,1147806313503698944,1147806313503698944,...,"{'iso_language_code': 'en', 'result_type': 're...",None,0.0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is why Golden Retrievers are one of the f...,False,"{'id': 1054861426106413057, 'id_str': '1054861..."
1,NaN,NaN,2019-07-07 01:19:55,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,NaN,1147676584243056641,1147676584243056640,...,"{'iso_language_code': 'en', 'result_type': 're...",None,0.0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This handyman doesn’t seem to know what he’s d...,True,"{'id': 832457747899654144, 'id_str': '83245774..."
2,NaN,NaN,2019-07-06 21:18:59,"{'hashtags': [{'text': 'goldenretriever', 'ind...","{'media': [{'id': 1147615936343683072, 'id_str...",15,False,NaN,1147615952122601472,1147615952122601472,...,"{'iso_language_code': 'en', 'result_type': 're...",None,0.0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",New Fren!! #goldenretriever #weratedogs #dogso...,False,"{'id': 3764374463, 'id_str': '3764374463', 'na..."
3,NaN,NaN,2019-07-06 09:34:18,{'hashtags': [{'text': 'rescuedogsofinstagram'...,NaN,1,False,NaN,1147438615435386880,1147438615435386880,...,"{'iso_language_code': 'en', 'result_type': 're...",None,0.0,0,False,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",Who is all that and a bag of chips? That would...,True,"{'id': 60259880, 'id_str': '60259880', 'name':..."
4,NaN,NaN,2019-07-06 05:39:42,"{'hashtags': [{'text': 'WeRateDogs', 'indices'...","{'media': [{'id': 1147379566119276544, 'id_str...",0,False,NaN,1147379574721806336,1147379574721806336,...,"{'iso_language_code': 'en', 'result_type': 're...",None,0.0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is Milo and he is now and 8 year old pupp...,False,"{'id': 1139405949519552512, 'id_str': '1139405..."


## Assessing

For the assessing we will follow these steps:

1. Check for missing data
2. Check for problems with the data structure
3. Check for quality issues

In [22]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [23]:
df_tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB
